In [1]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
import numpy as np

from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [3]:
def get_n(z, dens, mass_mol):
    '''
    This functions calculates the electron density for a
    given metal
    
    Inputs:
    z: integer for number of valence electrons
    dens: float for density of metal in kg/m**3
    mass_mol: float for molar mass in kg
    
    Returns:
    n: float for electron density in m**-3
    '''
    
    n = z*Na * dens/mass_mol
    return n

In [4]:
def get_Ef(n):
    '''
    This function calculates the Fermi energy (in eV) for
    a given metal with electron density n.
    
    Inputs:
    n: float for electron density in m**-3
    
    Returns:
    Ef: float for Fermi energy in eV
    '''
    
    Ef = h**2/(2*me)*(3*pi**2*n)**(2.0/3.0)
    return Ef

In [5]:
def get_mu(Ef,T):
    '''
    This function calculates the Fermi Level (chemical potential) for
    a metal characterized by Fermi energy Ef at temperature T. This
    calculations is done by applying second order Sommerfeld expansion.
    
    Inputs:
    Ef: float for Fermi energy in eV
    T: float for temperature in 10^4*K
    
    Returns:
    mu: float for Fermi level in eV
    '''
    
    mu = Ef*(1.0-1.0/3.0*(pi*kb*T/2.0/Ef)**2)
    return mu

In [6]:
def get_g(E):
    '''
    This functions calculates the Density of States (DoS) for a given
    energy, in the free electron gas model.
    
    Inputs:
    E: float for energy in eV
    
    Returns:
    g: number of states for unit of energy and volume (in eV**-1 A**-3)
    '''
    
    g = me/h**2/pi**2 * np.sqrt(2*me*E/h**2) * 1.0e-30
    return g

In [7]:
def calculate_integrals(E_values, occupancy_values):
    '''
    This function calculates total energy (for unit of volume)
    and total electron number (for unit of volume) by applying
    numerical integration to a given energy range and occupancy.
    '''
    
    n_of_elec = 0.0
    U_total = 0.0
    de = (max(E_values) - min(E_values))/pts
    for i in range(pts):
        E = E_values[i]
        occu = occupancy_values[i]
        n_of_elec = n_of_elec + occu*de
        U_total = U_total + E*occu*de
    return n_of_elec, U_total

In [8]:
def get_U(Ef, T):
    '''
    XXXThis function updates the temperature-dependent marks
    when temperature (or metal) is changed.
    '''

    # Calculate mu
    mu = get_mu(Ef,T)

    # Calculate F-D distribution and occupancy
    y_values = np.empty(pts)
    occupancy_values = np.empty(pts)
    overflow = False  # Flag to control overflow with very little exponents
    
    for i in range(pts):
        if overflow == False:
            E = E_values[i]
            y_values[i] = 1.0/(np.exp((E-mu)/kb/T)+1.0)
            if y_values[i] < 1.0e-20:  # Exponents smaller than -20 considered 0
                overflow = True
        else:
            y_values[i] = 0.0
        occupancy_values[i] = y_values[i] * g_values[i]

    
    # Calculate mean energy and electron density
    n_of_electrons, U_total = calculate_integrals(E_values, occupancy_values)    
    u_absolute = U_total / n_of_electrons
    u_relative = u_absolute / Ef

    return u_absolute

In [9]:
def update_metal(change):
    '''
    XXXThis functions updates both figures when
    the current metal is changed.
    '''
    
    # Read widgets
    metal = metal_dropdown.value
    
    # Calculate Ef and Tf
    rho = metal_densities[metal]
    mass = molar_masses[metal]
    z = valence_numbers[metal]  
    n = get_n(z, rho, mass)
    Ef = get_Ef(n)
    Tf = Ef/kb
   
    # Get energies
    U_values = np.empty(pts_T)
    for i in range(pts_T):
        T = T_values[i]
        U_values[i] = get_U(Ef,T)
        
        
    # Get Cv
    Cv_values = np.empty(pts_T)
    dT = (T_max-T_min) / pts_T
    for i in range(pts_T-1):
        Cv_values[i] = (U_values[i+1] - U_values[i]) / dT
    Cv_values[pts_T-1] = 1.5
    Cv_values = Cv_values/kb
    
    # Update curves
    UPlot_001.y = U_values/Ef
    CvPlot_002.y = Cv_values

In [10]:
#######################
###   PARAMETERS    ###
#######################

# Universal constant (only used to get Ef and g)
h = 6.5821e-16 #Planck's reduced constant in eV*s
c = 3.0e8 # Speed of Light in m/s
me = 510998.9/c**2 # Electron mass in eV/c**2
Na = 6.0221e23
kb = 8.617e-1 # Boltzmann's in eV/(10**4 K).
pi =  np.pi

# Plot parameters (temperatures in 10**4 K)
pts = 20000 # Number of points to calculate
pts_T = 500 # XXX
T_min = 0.001 # Minimun T value on slider
T_max = 20.0 # Maximun T value on slider

# Metal data:(data obtained from Wikipedia)
metal_labels = ["Sodioa", "Aluminioa", "Burdina", "Kobrea"] 
metal_densities = [968.0, 2700.0, 7874.0, 8960.0] # In kg/m**3
molar_masses = [0.0229, 0.026, 0.0558, 0.0635] # In Kg
valence_numbers = [1, 3, 2, 1]

# Initial values
T = 2.0  # Initial T value
metal = 1 # Initial metal index {0: 'Na', 1: 'Al', 2: 'Fe', 3: 'Cu'}

# Figures xRanges
T_values = np.linspace(T_min, T_max, pts_T)

# E and g values
E_values = np.linspace(0.0, 200.0, pts)
g_values = np.empty(pts)
for i in range(pts):
    E = E_values[i]
    g_values[i] = get_g(E)


########################
###CREATE THE FIGURES###
########################

fig_134_001 = bq.Figure(title='Elektroiko energia tenperaturarekiko',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-right',
                legend_style= {'fill': 'white', 'stroke': 'grey'},
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                layout = widgets.Layout(width='50%'),
                toolbar = True,
    )

fig_134_002 = bq.Figure(title='Bero ahalmena tenperaturarekiko',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-right',
                legend_style= {'fill': 'white', 'stroke': 'grey'},
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                layout = widgets.Layout(width='50%'),
                toolbar = True,
    )

scale_x_134_001 = bqs.LinearScale(min = 0.0, max = T_max)
scale_y_134_001 = bqs.LinearScale(min = 0.5, max = 2)

scale_x_134_002 = bqs.LinearScale(min = 0.0, max = T_max)
scale_y_134_002 = bqs.LinearScale(min = 0.0, max = 1.5)


axis_x_134_001 = bqa.Axis(scale=scale_x_134_001,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                #tick_values = np.linspace(p_min, p_max, 7),
                num_ticks=3,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='T',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y_134_001 = bqa.Axis(
                scale=scale_y_134_001,
                tick_format='1.3E',#'0.2f',
                tick_style={'font-size': '15px'},
                tick_values=np.linspace(0.5,2.0,7),
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='U',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='60px')

axis_x_134_002 = bqa.Axis(scale=scale_x_134_002,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                #tick_values = np.linspace(p_min, p_max, 7),
                num_ticks=3,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='T',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y_134_002 = bqa.Axis(
                scale=scale_y_134_002,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=6,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='Cv',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')


fig_134_001.axes = [axis_x_134_001, axis_y_134_001]
fig_134_002.axes = [axis_x_134_002, axis_y_134_002]


########################
####CREATE THE MARKS####
########################

UPlot_001 = bqm.Lines(
            x = T_values, 
            y = [], 
            scales = {'x': scale_x_134_001, 'y': scale_y_134_001}, 
            opacities = [1.0],
            visible = True, #True, #t == '1.00',
            colors = ['blue'],
            labels = ['Enegia'],
            display_legend = True
)

CvPlot_002 = bqm.Lines(
            x = T_values,
            y = [],
            scales = {'x': scale_x_134_002, 'y': scale_y_134_002},
            opacities = [1.0],
            visible = True,
            colors = ['green'],
            labels = ['Bero ahalmena'],
            display_legend = True,
)

fig_134_001.marks = [UPlot_001]
fig_134_002.marks = [CvPlot_002]


########################
######  WIDGETS  #######
########################

## Input Widgets ##

metal_dropdown = widgets.Dropdown(
    options=[('Sodioa', 0), ('Aluminioa', 1), ('Burdina', 2), ('Kobrea', 3)],
    value=metal,
    description='Gasa:',
    disabled=False,
)

metal_dropdown.observe(update_metal, 'value')

T_slider = widgets.FloatSlider(
    min=T_min,
    max=T_max,
    step=0.5,
    value=T,
    description='T',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width='90%'),
)


########################
###  INIT FIGURES  ####
########################

update_metal(None) # Read widgets values and assign marks' x,y values


########################
######  LAYOUT  ########
########################

widgets.HBox([metal_dropdown,fig_134_001,fig_134_002])


[ 7.17947373  7.18034653  7.18109653  7.18233824  7.18407152  7.18629617
  7.18901192  7.19221844  7.19591536  7.20010222  7.20477852  7.2099437
  7.21559711  7.22173808  7.22836583  7.23547954  7.24307832  7.25116119
  7.25972713  7.26877502  7.27830367  7.28831182  7.2987981   7.30976108
  7.32119922  7.33311091  7.3454944   7.35834788  7.37166939  7.3854569
  7.39970822  7.41442109  7.42959309  7.4452217   7.46130427  7.47783804
  7.49482011  7.51224746  7.53011698  7.54842541  7.5671694   7.58634548
  7.60595008  7.62597952  7.64643004  7.66729778  7.6885788   7.71026908
  7.73236453  7.75486098  7.77775421  7.80103994  7.82471385  7.84877154
  7.87320862  7.89802062  7.92320307  7.94875148  7.97466131  8.00092803
  8.0275471   8.05451397  8.08182407  8.10947287  8.13745581  8.16576837
  8.19440601  8.22336423  8.25263855  8.2822245   8.31211763  8.34231354
  8.37280782  8.40359614  8.43467416  8.46603759  8.49768219  8.52960374
  8.56179806  8.59426102  8.62698852  8.65997651  8.6